# Analisis exploratorio de las variables en relación a los Spoilers en comentarios de IMDB

## 1. Procesamiento de datos

In [1]:
# Importamos las librerias generales para el proyecto
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Lectura de los datos iniciales
movies_details = pd.read_json("./data/movies/IMDB_movie_details.json", lines=True)
movies_reviews = pd.read_json("./data/movies/IMDB_reviews.json", lines=True)
movies_info = pd.read_csv('./data/movies_info/IMDb movies.csv')

D:\ProgramData\Anaconda3\envs\data_science\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# Eliminamos columnas inncesarias o repetidas para hacer 
movies_details_pre = movies_details[['movie_id', 'rating','release_date']]
movies_info_pre = movies_info[['imdb_title_id', 'original_title',
       'genre', 'duration', 'country', 'language', 'director', 'writer',
       'production_company', 'actors', 'votes',
       'budget', 'usa_gross_income', 'worlwide_gross_income', 'metascore',
       'reviews_from_users', 'reviews_from_critics']]

In [4]:
# Combinamos los datasets para tener los datos completos
movies_data = movies_details_pre.merge(movies_info_pre, right_on="imdb_title_id", left_on="movie_id")

# Corregimos errores de repetición y de formato de los valores
movies_data = movies_data.drop(columns=["imdb_title_id"])
movies_data["budget"] = movies_data["budget"].str.split(" ", n = 1, expand = True)[1]
movies_data["budget"] = movies_data["budget"].astype('float64')
movies_data["usa_gross_income"] = movies_data["usa_gross_income"].str.split(" ", n = 1, expand = True)[1]
movies_data["usa_gross_income"] = movies_data["usa_gross_income"].astype('float64')
movies_data["worlwide_gross_income"] = movies_data["worlwide_gross_income"].str.split(" ", n = 1, expand = True)[1]
movies_data["worlwide_gross_income"] = movies_data["worlwide_gross_income"].astype('float64')

In [5]:
# Revisamos la existencia de valores NaN en las distintas columnas
movies_data.isnull().any()

movie_id                 False
rating                   False
release_date             False
original_title           False
genre                    False
duration                 False
country                  False
language                 False
director                 False
writer                   False
production_company       False
actors                   False
votes                    False
budget                    True
usa_gross_income          True
worlwide_gross_income     True
metascore                 True
reviews_from_users        True
reviews_from_critics     False
dtype: bool

In [6]:
# Comprobamos la forma de los datos para rellenar los valores NaN
movies_data.describe()

,rating,duration,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
count,1534.000000,1534.000000,1.534000e+03,1.436000e+03,1.476000e+03,1.517000e+03,1431.000000,1533.000000,1534.000000
mean,7.049022,116.722947,2.647611e+05,6.497236e+07,9.017659e+07,2.018458e+08,62.636618,658.522505,224.512386
std,0.956515,22.759115,2.682681e+05,2.763739e+08,9.706665e+07,2.561669e+08,17.657524,676.416985,162.416418
min,2.400000,67.000000,1.413000e+03,6.000000e+00,5.090000e+02,5.760000e+02,9.000000,5.000000,1.000000
25%,6.400000,101.000000,9.080425e+04,1.575000e+07,2.556240e+07,3.528179e+07,50.000000,256.000000,106.250000
50%,7.100000,113.000000,1.863885e+05,3.500000e+07,5.840391e+07,1.159009e+08,63.000000,468.000000,178.000000
75%,7.800000,128.000000,3.455498e+05,7.925000e+07,1.242690e+08,2.647710e+08,75.000000,828.000000,311.750000
max,9.500000,321.000000,2.278845e+06,1.000000e+10,9.366622e+08,2.790439e+09,100.000000,8232.000000,999.000000


In [7]:
# Rellenamos los valores con la mediana para que no se vean afectados por los "outliers" y comprobamos nuevamente
movies_data = movies_data.fillna(movies_data.median())
movies_data.isnull().any()

C:\Users\ALEJAN~1\AppData\Local\Temp/ipykernel_6136/2716236179.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  movies_data = movies_data.fillna(movies_data.median())


movie_id                 False
rating                   False
release_date             False
original_title           False
genre                    False
duration                 False
country                  False
language                 False
director                 False
writer                   False
production_company       False
actors                   False
votes                    False
budget                   False
usa_gross_income         False
worlwide_gross_income    False
metascore                False
reviews_from_users       False
reviews_from_critics     False
dtype: bool